In [1]:
from tqdm import tqdm
import sys
import os
import nltk
nltk.download('punkt')  
from nltk.translate.bleu_score import corpus_bleu
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("blue_score.ipynb"), "..")))
from datasets import load_dataset

from transformers_models.marian.marianMT import MarianMt

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:

def retrieve_data():
    dataset = load_dataset("wmt14", "de-en")
    
    subset = dataset["train"].select(range(120000))

    train_data = subset.select(range(100000))
    val_data = subset.select(range(100000, 110000))
    test_data = subset.select(range(110000, 120000))

    return {
        "train": train_data,
        "validation": val_data,
        "test": test_data
    }


In [10]:
def translated(n, model):
    return model.translate_text(n)

In [14]:

data = retrieve_data()


In [20]:
data["validation"]["translation"]

[{'de': 'Ich habe es mit Erfolg in Belgien als zunächst einzigem Land in Europa, dem später Frankreich folgte, eingeführt.',
  'en': 'I successfully introduced it in Belgium as the only country in Europe, and it was soon copied in France.'},
 {'de': 'Meiner Ansicht stellt es einer der wenigen praktischen Wege dar, die heute bestehen.',
  'en': 'I believe it is one of the few practical options which exist at the moment.'},
 {'de': 'Ich unterstütze den Bericht Karamanou.',
  'en': 'I support the Karamanou report.'},
 {'de': 'Gratulation!', 'en': 'Congratulations.'},
 {'de': 'Herr Präsident, Frau Kommissarin! Auch wenn sich die Integration der Frauen hinsichtlich Ausbildung, Beschäftigung und Politik verbessert hat - und sich weiterhin verbessert -, sind wir zweifellos mit erheblichen Problemen und gravierenden Ungleichheiten in den angesprochenen gesellschaftlichen Bereichen konfrontiert.',
  'en': 'Mr President, Commissioner, despite the fact that the number of women in education, emplo

In [19]:
data["test"]["translation"]

[{'de': 'Das hindert mich aber nicht daran, diesen Änderungsantrag aus den von mir dargelegten Gründen, die sowohl technischer als auch politischer Natur sind, abzulehnen.',
  'en': 'But that does not prevent me from disagreeing with this specific amendment for the reasons stated, reasons which are of a technical and a political nature.'},
 {'de': 'Herr Präsident! Der Vorschlag für eine Verordnung des Rates über die Hilfe für die Türkei im Rahmen der Heranführungsstrategie und der Bericht unseres Kollegen Swoboda verkörpern einen weiteren bedeutenden Moment auf dem Weg zu einem großen historischen Ziel.',
  'en': 'Mr President, the proposal for a Council regulation on assistance to Turkey under the pre-accession strategy and the report by Mr Swoboda mark yet another significant step towards achieving a great historical objective.'},
 {'de': 'Die wirtschaftliche Verflechtung zwischen der Türkei und der Europäischen Union hat ja nun schon einen erheblichen Umfang erreicht, da immerhin me

In [18]:
data["train"]["translation"]

[{'de': 'Wiederaufnahme der Sitzungsperiode',
  'en': 'Resumption of the session'},
 {'de': 'Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie schöne Ferien hatten.',
  'en': 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.'},
 {'de': 'Wie Sie feststellen konnten, ist der gefürchtete "Millenium-Bug " nicht eingetreten. Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden.',
  'en': "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful."},
 {'de': 'Im Parlament besteht der Wunsch nach einer Aussprache im

In [12]:

model = MarianMt("Helsinki-NLP/opus-mt-en-de")

In [ ]:
english = [n["en"] for n in data["train"]["translation"]]
german = [n["de"] for n in data["train"]["translation"]]

In [27]:
english

['Resumption of the session',
 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.',
 "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.",
 'You have requested a debate on this subject in the course of the next few days, during this part-session.',
 "In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union.",
 "Please rise, then, for this minute' s silence.",
 "(The House rose and observed a minute' s silence)",
 'Madam President, on a point of order.',
 'You will be aware from the press and television that there have be

In [28]:
translations = model.batch_translate(english)

print(translations)


Translating:   0%|          | 1/3125 [01:02<54:11:34, 62.45s/it]

Batch 1 / 3126


Translating:   0%|          | 1/3125 [01:29<77:45:21, 89.60s/it]


KeyboardInterrupt: 

In [ ]:
list_of_references = []
hypotheses = []

for ref, trans in zip(german, translations):

    ref_tokens = nltk.word_tokenize(ref.lower())
    trans_tokens = nltk.word_tokenize(trans.lower())

    list_of_references.append([ref_tokens])
    
    hypotheses.append(trans_tokens)


In [8]:
score = round(corpus_bleu(list_of_references, hypotheses), 3)
print("Corpus BLEU:", score)  
print("Corpus BLEU (%):", score * 100)


Corpus BLEU: 0.249
Corpus BLEU (%): 24.9
